# NVIDIA Trellis NIM Complete Deployment and Usage Guide

This notebook provides a complete end-to-end solution for deploying and using NVIDIA Trellis NIM on an L40s GPU, including:

- GPU availability check with nvidia-smi
- Complete NIM deployment following NVIDIA's official blueprint
- Image-to-3D generation workflow
- Integration with the Lovable UI

## Prerequisites

- NVIDIA GPU instance with at least one **L40S** GPU (tested on Ubuntu 22.04)
- An [NVIDIA Developer](https://developer.nvidia.com/) account with access to [NGC](https://ngc.nvidia.com/)
- NGC API key generated from your NGC account
- Docker 24+ with NVIDIA container runtime
- Internet connectivity for downloading the NIM container

**Note:** This notebook should be run on the GPU instance where you want to deploy the NIM container.


## Step 1: Check GPU Availability

First, let's verify that we have access to an L40s GPU and that the NVIDIA drivers are properly installed.


In [ ]:
# Check GPU availability and specifications
import subprocess
import sys
import os

def run_command(cmd, description=""):
    """Run a shell command and return the result"""
    try:
        print(f"Running: {cmd}")
        if description:
            print(f"Description: {description}")
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, check=True)
        print(result.stdout)
        if result.stderr:
            print(f"Warnings/Info: {result.stderr}")
        return result.stdout
    except subprocess.CalledProcessError as e:
        print(f"Error running command: {e}")
        print(f"Error output: {e.stderr}")
        return None

# Check NVIDIA driver and GPU information
print("=== Checking GPU Availability ===")
gpu_info = run_command("nvidia-smi", "Check NVIDIA GPU status and driver version")

if gpu_info and "L40S" in gpu_info:
    print("✅ L40S GPU detected!")
elif gpu_info:
    print("⚠️  GPU detected but may not be L40S. Proceeding anyway...")
    print("Detected GPUs:")
    print(gpu_info)
else:
    print("❌ No NVIDIA GPU detected or drivers not installed!")
    print("Please ensure:")
    print("1. NVIDIA drivers are installed")
    print("2. You're running on a GPU-enabled instance")
    print("3. The instance has an L40S GPU")
    sys.exit(1)


## Step 2: System Preparation and Docker Setup

Next, we'll ensure Docker is installed with the NVIDIA Container Toolkit for GPU passthrough into containers.


In [ ]:
# Check and install Docker if needed
print("=== Checking Docker Installation ===")

# Check if Docker is installed
docker_version = run_command("docker --version", "Check Docker version")

if not docker_version:
    print("Docker not found. Installing Docker...")
    # Update packages
    run_command("sudo apt update && sudo apt upgrade -y", "Update system packages")
    run_command("sudo apt install -y build-essential git curl wget python3-pip", "Install required tools")
    
    # Install Docker
    run_command("curl -fsSL https://get.docker.com | sudo sh", "Install Docker")
    run_command("sudo systemctl enable docker --now", "Enable Docker service")
    
    # Add current user to docker group (requires re-login to take effect)
    run_command("sudo usermod -aG docker $USER", "Add user to docker group")
    
    print("✅ Docker installed successfully!")
    print("Note: You may need to restart your session for docker group membership to take effect.")
else:
    print("✅ Docker is already installed")

# Check if NVIDIA Container Toolkit is installed
print("\n=== Checking NVIDIA Container Toolkit ===")
toolkit_check = run_command("which nvidia-ctk", "Check if nvidia-ctk is available")

if not toolkit_check:
    print("Installing NVIDIA Container Toolkit...")
    
    # Install NVIDIA Container Toolkit
    commands = [
        'distribution=$(. /etc/os-release;echo $ID$VERSION_ID)',
        'curl -s -L https://nvidia.github.io/libnvidia-container/gpgkey | sudo gpg --dearmor -o /usr/share/keyrings/nvidia-container-toolkit-keyring.gpg',
        'curl -s -L https://nvidia.github.io/libnvidia-container/$distribution/libnvidia-container.list | sed \'s#deb https://#deb [signed-by=/usr/share/keyrings/nvidia-container-toolkit-keyring.gpg] https://#g\' | sudo tee /etc/apt/sources.list.d/nvidia-container-toolkit.list',
        'sudo apt update',
        'sudo apt install -y nvidia-container-toolkit',
        'sudo nvidia-ctk runtime configure --runtime=docker',
        'sudo systemctl restart docker'
    ]
    
    for cmd in commands:
        run_command(cmd, f"NVIDIA Container Toolkit setup step")
    
    print("✅ NVIDIA Container Toolkit installed successfully!")
else:
    print("✅ NVIDIA Container Toolkit is already installed")

# Test Docker with GPU access
print("\n=== Testing Docker GPU Access ===")
gpu_test = run_command("sudo docker run --rm --gpus all nvidia/cuda:11.8-base-ubuntu20.04 nvidia-smi", 
                      "Test Docker GPU passthrough")

if gpu_test and "Driver Version" in gpu_test:
    print("✅ Docker can successfully access GPUs!")
else:
    print("❌ Docker GPU access test failed!")
    print("This might require a system restart or Docker service restart.")


## Step 3: NGC Authentication and NIM Deployment

Now we'll authenticate with NVIDIA NGC and deploy the Trellis NIM container following NVIDIA's official blueprint from https://build.nvidia.com/microsoft/trellis/deploy


In [ ]:
import getpass
import time
import threading

# Get NGC API key from user
print("=== NGC Authentication Setup ===")
print("You'll need your NGC API key from https://ngc.nvidia.com/setup/api-key")
print("When prompted for Docker login username, use: $oauthtoken")

# Check if NGC_API_KEY is already set in environment
ngc_api_key = os.environ.get('NGC_API_KEY')

if not ngc_api_key:
    ngc_api_key = getpass.getpass("Enter your NGC API key: ")
    os.environ['NGC_API_KEY'] = ngc_api_key
else:
    print("✅ NGC API key found in environment")

# Authenticate with NGC registry
print("\n=== Authenticating with NGC Registry ===")
print("Logging into nvcr.io...")

# Use expect-like approach for Docker login
login_cmd = f'echo "{ngc_api_key}" | sudo docker login nvcr.io --username \'$oauthtoken\' --password-stdin'
login_result = run_command(login_cmd, "Login to NGC registry")

if login_result and "Login Succeeded" in login_result:
    print("✅ Successfully authenticated with NGC registry")
else:
    print("❌ Failed to authenticate with NGC registry")
    print("Please check your NGC API key and try again")
    sys.exit(1)


In [ ]:
# Deploy Trellis NIM Container according to NVIDIA Blueprint
print("=== Deploying NVIDIA Trellis NIM Container ===")
print("This will pull and start the Trellis NIM container on port 8000")
print("According to: https://build.nvidia.com/microsoft/trellis/deploy")

# Check if container is already running
existing_container = run_command("sudo docker ps --filter 'name=trellis-nim' --format '{{.Names}}'", 
                                "Check for existing Trellis NIM container")

if existing_container and "trellis-nim" in existing_container:
    print("⚠️  Trellis NIM container is already running")
    user_input = input("Do you want to stop and restart it? (y/N): ").strip().lower()
    if user_input == 'y':
        run_command("sudo docker stop trellis-nim", "Stop existing container")
        run_command("sudo docker rm trellis-nim", "Remove existing container")
    else:
        print("Keeping existing container running")
else:
    print("No existing Trellis NIM container found")

# Pull the latest Trellis NIM image
print("\n=== Pulling Trellis NIM Image ===")
print("This may take several minutes depending on your internet connection...")

pull_result = run_command("sudo docker pull nvcr.io/nim/microsoft/trellis:latest", 
                         "Pull Trellis NIM container image")

if not pull_result or "Error" in str(pull_result):
    print("❌ Failed to pull Trellis NIM image")
    sys.exit(1)
else:
    print("✅ Successfully pulled Trellis NIM image")

# Start the Trellis NIM container
print("\n=== Starting Trellis NIM Container ===")
print("Starting container on port 8000 with GPU access...")

# Following the exact blueprint from NVIDIA
nim_cmd = f"""sudo docker run -d \\
    --gpus all \\
    -e NGC_API_KEY={ngc_api_key} \\
    -p 8000:8000 \\
    --name trellis-nim \\
    nvcr.io/nim/microsoft/trellis:latest"""

start_result = run_command(nim_cmd, "Start Trellis NIM container")

if start_result:
    print("✅ Trellis NIM container started successfully!")
    print(f"Container ID: {start_result.strip()}")
    
    # Wait for container to be ready
    print("\nWaiting for container to start up...")
    time.sleep(10)
    
    # Check container status
    status_result = run_command("sudo docker ps --filter 'name=trellis-nim'", "Check container status")
    print("Container status:")
    print(status_result)
    
    # Check container logs
    print("\nChecking container startup logs:")
    logs_result = run_command("sudo docker logs trellis-nim --tail 20", "Get container logs")
    print(logs_result)
    
else:
    print("❌ Failed to start Trellis NIM container")
    sys.exit(1)


## Step 4: Test NIM API Connectivity

Let's verify that the NIM container is running and responding to API requests.


In [ ]:
# Install required Python packages for API testing
print("=== Installing Python Dependencies ===")
run_command("pip install requests python-dotenv pillow tqdm ipywidgets", "Install required Python packages")

import requests
import json
from pathlib import Path
import base64
from tqdm.auto import tqdm

# Configure NIM endpoints
NIM_BASE_URL = "http://localhost:8000"
NIM_JOB_PATH = "/v1/images-to-3d"
BASE_URL = NIM_BASE_URL.rstrip('/')
JOB_ROOT = f"{BASE_URL}{NIM_JOB_PATH}".rstrip('/')
SUBMIT_ENDPOINT = f"{JOB_ROOT}/jobs"
STATUS_ENDPOINT = f"{JOB_ROOT}/jobs/{{job_id}}"

HEADERS = {'Content-Type': 'application/json'}
if ngc_api_key:
    HEADERS['Authorization'] = f'Bearer {ngc_api_key}'

print(f"Submit endpoint: {SUBMIT_ENDPOINT}")
print(f"Status endpoint: {STATUS_ENDPOINT}")

# Test API connectivity
print("\n=== Testing NIM API Connectivity ===")

# Wait for the container to fully start up
print("Waiting for NIM container to fully initialize...")
max_retries = 30
retry_count = 0

while retry_count < max_retries:
    try:
        # Try to make a simple request to check if the API is up
        response = requests.get(f"{BASE_URL}/health", timeout=10)
        if response.status_code == 200:
            print("✅ NIM API is responding!")
            break
    except requests.exceptions.RequestException as e:
        pass
    
    retry_count += 1
    print(f"Waiting... (attempt {retry_count}/{max_retries})")
    time.sleep(10)

if retry_count >= max_retries:
    print("❌ NIM API is not responding after waiting")
    print("Checking container logs for errors:")
    run_command("sudo docker logs trellis-nim --tail 50", "Get detailed container logs")
    
    # Try to get more information about the container
    run_command("sudo docker inspect trellis-nim", "Inspect container details")
else:
    print("✅ NIM container is ready for image-to-3D generation!")
    
    # Get some basic info about the NIM service
    try:
        response = requests.get(f"{BASE_URL}/v1/models", headers=HEADERS, timeout=30)
        if response.status_code == 200:
            models_info = response.json()
            print(f"Available models: {models_info}")
        else:
            print(f"Models endpoint returned status: {response.status_code}")
    except Exception as e:
        print(f"Could not fetch models info: {e}")
        
print("\n=== Deployment Summary ===")
print("✅ GPU detected and accessible")
print("✅ Docker and NVIDIA Container Toolkit installed")  
print("✅ NGC authentication successful")
print("✅ Trellis NIM container deployed and running")
print("✅ API connectivity verified")
print(f"🌐 NIM API available at: {BASE_URL}")
print("📝 Ready for image-to-3D generation!")


## Step 5: Image-to-3D Generation Workflow

Now let's test the complete image-to-3D generation workflow using the deployed NIM container.


In [ ]:
# Test the image-to-3D generation with a sample workflow
import io
import requests
import json
import base64
import time
from pathlib import Path
from tqdm.auto import tqdm

def create_sample_image():
    """Create a simple test image if no images are available"""
    try:
        from PIL import Image, ImageDraw
        
        # Create a simple 512x512 test image
        img = Image.new('RGB', (512, 512), color='white')
        draw = ImageDraw.Draw(img)
        
        # Draw a simple chair-like shape
        # Seat
        draw.rectangle([150, 250, 350, 300], fill='brown', outline='black', width=2)
        # Back
        draw.rectangle([160, 150, 180, 300], fill='brown', outline='black', width=2)
        draw.rectangle([320, 150, 340, 300], fill='brown', outline='black', width=2)
        draw.rectangle([160, 150, 340, 180], fill='brown', outline='black', width=2)
        # Legs
        draw.rectangle([160, 300, 180, 380], fill='brown', outline='black', width=2)
        draw.rectangle([320, 300, 340, 380], fill='brown', outline='black', width=2)
        draw.rectangle([160, 380, 180, 400], fill='brown', outline='black', width=2)
        draw.rectangle([320, 380, 340, 400], fill='brown', outline='black', width=2)
        
        # Save test image
        test_image_path = Path('test_chair.png')
        img.save(test_image_path)
        print(f"✅ Created test image: {test_image_path}")
        return test_image_path
        
    except ImportError:
        print("PIL not available. Please provide your own image file.")
        return None

def encode_image_file(image_path):
    """Encode an image file to base64"""
    with open(image_path, 'rb') as f:
        return base64.b64encode(f.read()).decode('utf-8')

def submit_generation_job(images_base64, **params):
    """Submit an image-to-3D generation job"""
    payload = {
        'images': images_base64,
        'meshFormat': params.get('meshFormat', 'glb'),
        'textureFormat': params.get('textureFormat', 'png'),
        'seed': params.get('seed', 0),
        'noTexture': params.get('noTexture', False),
        'slatCfgScale': params.get('slatCfgScale', 4.0),
        'ssCfgScale': params.get('ssCfgScale', 8.0),
        'slatSamplingSteps': params.get('slatSamplingSteps', 25),
        'ssSamplingSteps': params.get('ssSamplingSteps', 25),
    }
    
    response = requests.post(SUBMIT_ENDPOINT, headers=HEADERS, json=payload, timeout=120)
    response.raise_for_status()
    return response.json()

def poll_job_completion(job_id, timeout=900):
    """Poll for job completion with progress bar"""
    start_time = time.time()
    
    with tqdm(desc=f"Processing job {job_id}", unit="poll") as pbar:
        while time.time() - start_time < timeout:
            try:
                status_response = requests.get(
                    STATUS_ENDPOINT.format(job_id=job_id),
                    headers=HEADERS,
                    timeout=30
                )
                status_response.raise_for_status()
                result = status_response.json()
                
                status = result.get('status', 'unknown')
                pbar.set_postfix(status=status)
                
                if status == 'succeeded':
                    pbar.set_description("✅ Generation completed!")
                    return result
                elif status == 'failed':
                    pbar.set_description("❌ Generation failed!")
                    return result
                    
                time.sleep(5)
                pbar.update(1)
                
            except Exception as e:
                print(f"Error polling job: {e}")
                time.sleep(5)
    
    raise TimeoutError(f"Job {job_id} timed out after {timeout} seconds")

def download_assets(result, output_dir="outputs"):
    """Download generated assets"""
    output_path = Path(output_dir)
    output_path.mkdir(exist_ok=True)
    
    if not result.get('assets'):
        print("No assets to download")
        return []
    
    downloaded_files = []
    
    for asset in result['assets']:
        asset_url = asset['url']
        asset_type = asset['type']
        file_extension = Path(asset_url).suffix or '.bin'
        
        filename = f"{result.get('jobId', 'unknown')}_{asset_type}{file_extension}"
        file_path = output_path / filename
        
        try:
            print(f"Downloading {asset_type} asset...")
            asset_response = requests.get(asset_url, timeout=300)
            asset_response.raise_for_status()
            
            file_path.write_bytes(asset_response.content)
            file_size = len(asset_response.content) / (1024 * 1024)  # MB
            
            print(f"✅ Downloaded: {file_path} ({file_size:.2f} MB)")
            downloaded_files.append(file_path)
            
        except Exception as e:
            print(f"❌ Failed to download {asset_type}: {e}")
    
    return downloaded_files

# Run the complete workflow
print("=== Complete Image-to-3D Generation Workflow ===")

# Step 1: Prepare test image (or use your own)
test_image = create_sample_image()

if test_image and test_image.exists():
    # Step 2: Encode image
    print(f"\n=== Encoding Image: {test_image} ===")
    encoded_image = encode_image_file(test_image)
    print(f"✅ Image encoded (size: {len(encoded_image)} characters)")
    
    # Step 3: Submit generation job
    print("\n=== Submitting Generation Job ===")
    try:
        submission = submit_generation_job(
            images_base64=[encoded_image],
            meshFormat='glb',
            textureFormat='png',
            seed=42,  # For reproducible results
            slatCfgScale=4.0,
            ssCfgScale=8.0,
            slatSamplingSteps=25,
            ssSamplingSteps=25
        )
        
        job_id = submission['jobId']
        print(f"✅ Job submitted successfully!")
        print(f"Job ID: {job_id}")
        print(f"Status: {submission.get('status', 'unknown')}")
        
        # Step 4: Poll for completion
        print(f"\n=== Waiting for Job Completion ===")
        print("This may take 5-15 minutes depending on complexity...")
        
        final_result = poll_job_completion(job_id)
        
        print(f"\n=== Generation Results ===")
        print(f"Final Status: {final_result.get('status')}")
        if final_result.get('message'):
            print(f"Message: {final_result.get('message')}")
            
        # Step 5: Download assets
        if final_result.get('status') == 'succeeded':
            print(f"\n=== Downloading Generated Assets ===")
            downloaded_files = download_assets(final_result)
            
            print(f"\n🎉 SUCCESS! Generated 3D model from your image!")
            print(f"Generated files: {len(downloaded_files)}")
            for file_path in downloaded_files:
                print(f"  📁 {file_path}")
                
            # Show GLB file info
            glb_files = [f for f in downloaded_files if f.suffix.lower() == '.glb']
            if glb_files:
                print(f"\n📦 GLB Model Ready!")
                print(f"You can now use the GLB file in any 3D application:")
                for glb_file in glb_files:
                    print(f"  🎯 {glb_file}")
                    print(f"     Size: {glb_file.stat().st_size / (1024*1024):.2f} MB")
        else:
            print(f"❌ Generation failed: {final_result.get('message', 'Unknown error')}")
            
    except Exception as e:
        print(f"❌ Error during generation: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ No test image available. Please provide an image file to test the workflow.")

print(f"\n=== Workflow Complete ===")
print(f"🌐 NIM API is running at: {BASE_URL}")
print(f"📝 You can now integrate this workflow with any web UI!")


## Step 6: Lovable UI Integration

Now we'll create a simple web interface that connects to the deployed NIM for easy image-to-3D generation. This shows how to integrate the NIM with any web UI framework.


In [ ]:
# Create a simple web UI for image upload and 3D model generation
import os
from pathlib import Path

# Create the HTML template for the web UI
html_content = '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Trellis NIM - Image to 3D Model Generator</title>
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        body {
            font-family: 'Arial', sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 20px;
        }
        .container {
            background: white;
            border-radius: 20px;
            box-shadow: 0 20px 40px rgba(0,0,0,0.1);
            padding: 40px;
            max-width: 600px;
            width: 100%;
            text-align: center;
        }
        h1 {
            color: #333;
            margin-bottom: 10px;
            font-size: 2.5em;
        }
        .subtitle {
            color: #666;
            margin-bottom: 30px;
            font-size: 1.1em;
        }
        .upload-area {
            border: 3px dashed #ddd;
            border-radius: 15px;
            padding: 60px 20px;
            margin: 30px 0;
            transition: all 0.3s ease;
            cursor: pointer;
        }
        .upload-area:hover, .upload-area.drag-over {
            border-color: #667eea;
            background-color: #f8f9ff;
        }
        .upload-icon {
            font-size: 4em;
            color: #ddd;
            margin-bottom: 20px;
        }
        .upload-text {
            color: #666;
            font-size: 1.2em;
            margin-bottom: 10px;
        }
        .upload-subtext {
            color: #999;
            font-size: 0.9em;
        }
        #file-input {
            display: none;
        }
        .preview-container {
            margin: 20px 0;
            display: none;
        }
        .preview-image {
            max-width: 300px;
            max-height: 300px;
            border-radius: 10px;
            box-shadow: 0 10px 20px rgba(0,0,0,0.1);
        }
        .generate-btn {
            background: linear-gradient(135deg, #667eea, #764ba2);
            color: white;
            border: none;
            padding: 15px 40px;
            border-radius: 50px;
            font-size: 1.1em;
            cursor: pointer;
            transition: all 0.3s ease;
            margin-top: 20px;
            display: none;
        }
        .generate-btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 10px 20px rgba(102, 126, 234, 0.3);
        }
        .generate-btn:disabled {
            opacity: 0.6;
            cursor: not-allowed;
            transform: none;
        }
        .progress-container {
            margin: 30px 0;
            display: none;
        }
        .progress-bar {
            width: 100%;
            height: 10px;
            background: #f0f0f0;
            border-radius: 5px;
            overflow: hidden;
            margin: 10px 0;
        }
        .progress-fill {
            height: 100%;
            background: linear-gradient(90deg, #667eea, #764ba2);
            border-radius: 5px;
            width: 0%;
            transition: width 0.3s ease;
        }
        .status-text {
            color: #666;
            margin: 10px 0;
        }
        .result-container {
            margin: 30px 0;
            display: none;
        }
        .download-btn {
            background: #28a745;
            color: white;
            border: none;
            padding: 12px 30px;
            border-radius: 25px;
            font-size: 1em;
            cursor: pointer;
            margin: 10px;
            transition: all 0.3s ease;
        }
        .download-btn:hover {
            background: #218838;
            transform: translateY(-1px);
        }
        .error-message {
            background: #ffe6e6;
            color: #d63384;
            padding: 15px;
            border-radius: 10px;
            margin: 20px 0;
            display: none;
        }
        .success-message {
            background: #e6ffe6;
            color: #28a745;
            padding: 15px;
            border-radius: 10px;
            margin: 20px 0;
            display: none;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🎨 Trellis NIM</h1>
        <p class="subtitle">Transform your 2D images into stunning 3D models using NVIDIA's Trellis technology</p>
        
        <div class="upload-area" onclick="document.getElementById('file-input').click()">
            <div class="upload-icon">📤</div>
            <div class="upload-text">Click to upload or drag and drop</div>
            <div class="upload-subtext">PNG, JPG, JPEG, WEBP (max 10MB)</div>
        </div>
        
        <input type="file" id="file-input" accept="image/*" multiple>
        
        <div class="preview-container" id="preview-container">
            <h3>Selected Images:</h3>
            <div id="image-previews"></div>
        </div>
        
        <button class="generate-btn" id="generate-btn">Generate 3D Model</button>
        
        <div class="progress-container" id="progress-container">
            <div class="status-text" id="status-text">Preparing...</div>
            <div class="progress-bar">
                <div class="progress-fill" id="progress-fill"></div>
            </div>
        </div>
        
        <div class="error-message" id="error-message"></div>
        <div class="success-message" id="success-message"></div>
        
        <div class="result-container" id="result-container">
            <h3>🎉 Generation Complete!</h3>
            <p>Your 3D model has been generated successfully.</p>
            <div id="download-links"></div>
        </div>
    </div>

    <script>
        const fileInput = document.getElementById('file-input');
        const uploadArea = document.querySelector('.upload-area');
        const previewContainer = document.getElementById('preview-container');
        const imagePreviewsDiv = document.getElementById('image-previews');
        const generateBtn = document.getElementById('generate-btn');
        const progressContainer = document.getElementById('progress-container');
        const statusText = document.getElementById('status-text');
        const progressFill = document.getElementById('progress-fill');
        const errorMessage = document.getElementById('error-message');
        const successMessage = document.getElementById('success-message');
        const resultContainer = document.getElementById('result-container');
        const downloadLinksDiv = document.getElementById('download-links');
        
        let selectedFiles = [];
        const NIM_API_URL = 'http://localhost:8000/v1/images-to-3d';
        
        // Drag and drop functionality
        uploadArea.addEventListener('dragover', (e) => {
            e.preventDefault();
            uploadArea.classList.add('drag-over');
        });
        
        uploadArea.addEventListener('dragleave', () => {
            uploadArea.classList.remove('drag-over');
        });
        
        uploadArea.addEventListener('drop', (e) => {
            e.preventDefault();
            uploadArea.classList.remove('drag-over');
            handleFiles(e.dataTransfer.files);
        });
        
        fileInput.addEventListener('change', (e) => {
            handleFiles(e.target.files);
        });
        
        function handleFiles(files) {
            selectedFiles = Array.from(files).filter(file => 
                file.type.startsWith('image/') && file.size <= 10 * 1024 * 1024
            );
            
            if (selectedFiles.length === 0) {
                showError('Please select valid image files (max 10MB each)');
                return;
            }
            
            displayPreviews();
            generateBtn.style.display = 'inline-block';
            hideMessages();
        }
        
        function displayPreviews() {
            imagePreviewsDiv.innerHTML = '';
            
            selectedFiles.forEach((file, index) => {
                const reader = new FileReader();
                reader.onload = (e) => {
                    const img = document.createElement('img');
                    img.src = e.target.result;
                    img.className = 'preview-image';
                    img.style.margin = '10px';
                    imagePreviewsDiv.appendChild(img);
                };
                reader.readAsDataURL(file);
            });
            
            previewContainer.style.display = 'block';
        }
        
        generateBtn.addEventListener('click', async () => {
            if (selectedFiles.length === 0) return;
            
            generateBtn.disabled = true;
            progressContainer.style.display = 'block';
            resultContainer.style.display = 'none';
            hideMessages();
            
            try {
                // Convert images to base64
                statusText.textContent = 'Encoding images...';
                progressFill.style.width = '10%';
                
                const base64Images = await Promise.all(
                    selectedFiles.map(file => fileToBase64(file))
                );
                
                // Submit job
                statusText.textContent = 'Submitting generation job...';
                progressFill.style.width = '20%';
                
                const jobResponse = await fetch(`${NIM_API_URL}/jobs`, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        images: base64Images,
                        meshFormat: 'glb',
                        textureFormat: 'png',
                        seed: Math.floor(Math.random() * 1000000),
                        slatCfgScale: 4.0,
                        ssCfgScale: 8.0,
                        slatSamplingSteps: 25,
                        ssSamplingSteps: 25
                    })
                });
                
                if (!jobResponse.ok) {
                    throw new Error(`Failed to submit job: ${jobResponse.statusText}`);
                }
                
                const jobData = await jobResponse.json();
                const jobId = jobData.jobId;
                
                statusText.textContent = `Job submitted! ID: ${jobId}`;
                progressFill.style.width = '30%';
                
                // Poll for completion
                await pollJobCompletion(jobId);
                
            } catch (error) {
                console.error('Generation error:', error);
                showError(`Generation failed: ${error.message}`);
                progressContainer.style.display = 'none';
            } finally {
                generateBtn.disabled = false;
            }
        });
        
        async function fileToBase64(file) {
            return new Promise((resolve) => {
                const reader = new FileReader();
                reader.onload = () => {
                    const base64 = reader.result.split(',')[1];
                    resolve(base64);
                };
                reader.readAsDataURL(file);
            });
        }
        
        async function pollJobCompletion(jobId) {
            const maxAttempts = 180; // 15 minutes at 5-second intervals
            let attempts = 0;
            
            while (attempts < maxAttempts) {
                try {
                    const statusResponse = await fetch(`${NIM_API_URL}/jobs/${jobId}`);
                    const statusData = await statusResponse.json();
                    
                    const progress = Math.min(30 + (attempts / maxAttempts) * 60, 90);
                    progressFill.style.width = `${progress}%`;
                    
                    statusText.textContent = `Processing... Status: ${statusData.status}`;
                    
                    if (statusData.status === 'succeeded') {
                        progressFill.style.width = '100%';
                        statusText.textContent = 'Generation completed!';
                        showResults(statusData);
                        return;
                    } else if (statusData.status === 'failed') {
                        throw new Error(statusData.message || 'Generation failed');
                    }
                    
                    await new Promise(resolve => setTimeout(resolve, 5000));
                    attempts++;
                    
                } catch (error) {
                    throw new Error(`Polling failed: ${error.message}`);
                }
            }
            
            throw new Error('Generation timed out');
        }
        
        function showResults(resultData) {
            progressContainer.style.display = 'none';
            
            if (resultData.assets && resultData.assets.length > 0) {
                downloadLinksDiv.innerHTML = '';
                
                resultData.assets.forEach(asset => {
                    const downloadBtn = document.createElement('button');
                    downloadBtn.className = 'download-btn';
                    downloadBtn.textContent = `Download ${asset.type.toUpperCase()}`;
                    downloadBtn.onclick = () => {
                        const link = document.createElement('a');
                        link.href = asset.url;
                        link.download = `trellis_${asset.type}.${asset.type === 'mesh' ? 'glb' : 'png'}`;
                        link.click();
                    };
                    downloadLinksDiv.appendChild(downloadBtn);
                });
                
                resultContainer.style.display = 'block';
                showSuccess('3D model generated successfully! Click the buttons below to download your files.');
            } else {
                showError('No assets were generated. Please try again.');
            }
        }
        
        function showError(message) {
            errorMessage.textContent = message;
            errorMessage.style.display = 'block';
            successMessage.style.display = 'none';
        }
        
        function showSuccess(message) {
            successMessage.textContent = message;
            successMessage.style.display = 'block';
            errorMessage.style.display = 'none';
        }
        
        function hideMessages() {
            errorMessage.style.display = 'none';
            successMessage.style.display = 'none';
        }
    </script>
</body>
</html>'''

# Save the HTML file
html_file = Path('trellis_nim_ui.html')
html_file.write_text(html_content)

print("✅ Created Lovable UI for Trellis NIM!")
print(f"📁 Saved as: {html_file.absolute()}")
print()
print("🌐 To use the UI:")
print("1. Make sure your Trellis NIM container is running on http://localhost:8000")
print("2. Open the HTML file in your web browser:")
print(f"   file://{html_file.absolute()}")
print("3. Upload images and generate 3D models!")
print()
print("📝 Features of this UI:")
print("• Drag & drop image upload")
print("• Multiple image support")
print("• Real-time progress tracking")
print("• Direct GLB model downloads")
print("• Modern, responsive design")
print()
print("🔧 For production deployment:")
print("• Host the HTML file on a web server")
print("• Configure CORS on the NIM container if needed")
print("• Add HTTPS for secure file uploads")
print("• Customize styling and branding as needed")

# Also create a simple Python Flask server as an alternative
flask_server_code = '''#!/usr/bin/env python3
"""
Simple Flask server for Trellis NIM integration
Provides a web interface for image-to-3D generation

Usage:
    python3 trellis_flask_server.py

Then open: http://localhost:5000
"""

from flask import Flask, render_template_string, request, jsonify, send_file
import requests
import base64
import os
import tempfile
from pathlib import Path

app = Flask(__name__)
app.config['MAX_CONTENT_LENGTH'] = 50 * 1024 * 1024  # 50MB max file size

# Configuration
NIM_BASE_URL = "http://localhost:8000"
NIM_ENDPOINT = f"{NIM_BASE_URL}/v1/images-to-3d"

HTML_TEMPLATE = """''' + html_content.replace('http://localhost:8000/v1/images-to-3d', '/api') + '''"""

@app.route('/')
def index():
    return render_template_string(HTML_TEMPLATE)

@app.route('/api/jobs', methods=['POST'])
def submit_job():
    try:
        # Forward the request to the NIM container
        response = requests.post(
            f"{NIM_ENDPOINT}/jobs",
            json=request.json,
            headers={'Content-Type': 'application/json'},
            timeout=120
        )
        response.raise_for_status()
        return jsonify(response.json())
    except Exception as e:
        return jsonify({'error': str(e)}), 500

@app.route('/api/jobs/<job_id>', methods=['GET'])
def get_job_status(job_id):
    try:
        response = requests.get(
            f"{NIM_ENDPOINT}/jobs/{job_id}",
            timeout=30
        )
        response.raise_for_status()
        return jsonify(response.json())
    except Exception as e:
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    print("🚀 Starting Trellis NIM Flask Server...")
    print("📂 Make sure Trellis NIM is running at:", NIM_BASE_URL)
    print("🌐 Open your browser to: http://localhost:5000")
    print("⏹️  Press Ctrl+C to stop")
    
    app.run(host='0.0.0.0', port=5000, debug=True)
'''

# Save the Flask server
flask_file = Path('trellis_flask_server.py')
flask_file.write_text(flask_server_code)
os.chmod(flask_file, 0o755)

print(f"🐍 Also created Flask server: {flask_file.absolute()}")
print("   Install Flask: pip install flask")
print("   Run server: python3 trellis_flask_server.py")
print("   Access at: http://localhost:5000")
print()
print("🎯 Integration Complete!")
print("You now have two options for the Lovable UI integration:")
print("1. Static HTML file (simple, no server needed)")
print("2. Flask web server (more robust, handles CORS)")

# Show file structure
print(f"\n📁 Created Files:")
print(f"├── {html_file.name} (Static HTML UI)")
print(f"└── {flask_file.name} (Flask server)")
print(f"\n🌟 Your Trellis NIM deployment is now complete with UI integration!")
